In [ ]:
# Bibliotecas padrão
import os
from io import BytesIO
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

# Bibliotecas externas
import pandas as pd
import requests
from sqlalchemy import create_engine, inspect, text

# PySpark
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, StringType, FloatType
from pyspark.sql.window import Window
from pyspark.sql.functions import (
    col, lit, row_number, expr, monotonically_increasing_id, 
    regexp_replace, trim, lower, to_date, date_format, 
    dayofmonth, month, year, quarter, lag, when, sequence, 
    explode, count, sum as spark_sum
)

from pyspark.sql.functions import udf


In [ ]:
# Dicionários e variáveis
path_silver = "abfss://Hackathon_Emprega_Dados@onelake.dfs.fabric.microsoft.com/lakehouse_vendas.Lakehouse/Files/silver/"

In [ ]:
## Função para converter a coluna de data para o formato id_data
def convert_iddata(df, col_name):
    return pd.to_datetime(df[col_name], format="%Y%m%d").dt.strftime("%Y%m%d").astype("Int64")


In [ ]:
def classificar_despesa(item):
    if item in [
        'Despesas com matéria-prima',
        'Despesas de transporte e logística',
        'Mecanicos',
        'Manutenção de equipamentos'
    ]:
        return 'Custos Variáveis (CPV)'

    elif item in [
        'Despesas com marketing e publicidade',
        'Despesas com viagens e deslocamentos',
        'Despesas com telefonia móvel e fixa',
        'Trafego', 'Google Ads', 'Meta Ads', 'LinkedIn Ads', 'CRM', 'Landin page'
    ]:
        return 'Despesas Comerciais'

    elif item in [
        'Impostos e taxas'
    ]:
        return 'Impostos'

    elif item in [
        'Treinamento'
    ]:
        return 'Desenvolvimento Pessoal'

    else:
        return 'Despesas Administrativas'

In [ ]:
# Inicializa a SparkSession (se ainda não o fez)
spark = SparkSession.builder.appName("ClientesUnicos").getOrCreate()

# 1. Leitura das tabelas Delta da camada bronze
df_clientes_consig = (
    spark.read.format("delta")
    .load("Tables/b_vendas_clientes_consigcar")
    .selectExpr("Nome as nome_cliente", "WhatsApp as whatsapp")
    .distinct()
)
df_clientes_consig = df_clientes_consig.withColumn("empresa", lit("Consigcar"))

df_clientes_alu = (
    spark.read.format("delta")
    .load("Tables/b_vendas_clientes_alucar")
    .selectExpr("`Nome__Alucar_` as nome_cliente")
    .distinct()
)
df_clientes_alu = (
    df_clientes_alu.withColumn("whatsapp", lit(None).cast("string"))
    .withColumn("empresa", lit("Alucar"))
)

# 2. Combina as duas tabelas
df_clientes = df_clientes_consig.unionByName(df_clientes_alu)


# 3. Remover valores duplicados e adicionar 'id_cliente' na primeira posição

# Remove duplicatas primeiro
df_clientes_unicos = df_clientes.dropDuplicates(["nome_cliente"])

# Adiciona a coluna 'id_cliente'
window_spec = Window.orderBy("nome_cliente")
df_clientes_com_id = df_clientes_unicos.withColumn("id_cliente", row_number().over(window_spec) - 1)

# Pega todas as colunas existentes, exceto 'id_cliente' (se por algum motivo já existisse antes)
# Isso garante que não teremos duas colunas 'id_cliente' ao fazer o select final.
colunas_existentes = [col for col in df_clientes_com_id.columns if col != "id_cliente"]

# Cria a lista final de colunas com 'id_cliente' na primeira posição
colunas_finais = ["id_cliente"] + colunas_existentes

# Seleciona as colunas na ordem desejada
df_clientes_final = df_clientes_com_id.select(*colunas_finais).sort("id_cliente")

# 4. Salva a tabela na camada silver como Delta
df_clientes_final.write.format("delta").mode("overwrite").save("Tables/s_dim_cliente")


In [ ]:
# Leitura da tabela Delta na camada bronze
df_vend = (
    spark.read.format("delta")
    .load("Tables/b_vendas_clientes_consigcar")
    .select("Vendedor")
    .distinct()
    .withColumnRenamed("Vendedor", "nome_vendedor")
)

# Adiciona id_cliente único
df_vend = df_vend.withColumn("id_vendedor", monotonically_increasing_id())

colunas_existentes = [col for col in df_vend.columns if col != "id_vendedor"]

# Cria a lista final de colunas com 'id_cliente' na primeira posição
colunas_finais = ["id_vendedor"] + colunas_existentes

# Seleciona as colunas na ordem desejada
df_vendedor_final = df_vend.select(*colunas_finais).sort("id_vendedor")

# Salvar na camada silver como tabela Delta (substitui se já existir)
df_vendedor_final.write.format("delta").mode("overwrite").save("Tables/s_dim_vendedor")

In [ ]:
# Criar sessão Spark
spark = SparkSession.builder.getOrCreate()

# Gerar intervalo de datas de 2018-01-01 a 2030-12-31
start_date = datetime(2018, 1, 1)
end_date = datetime(2030, 12, 31)

date_list = []
current_date = start_date
while current_date <= end_date:
    date_list.append((int(current_date.strftime('%Y%m%d')), current_date))
    current_date += timedelta(days=1)

# Criar DataFrame Spark
df_tempo = spark.createDataFrame(date_list, ["id_data", "data"])

# Adicionar colunas
df_tempo = df_tempo.withColumn("dia", dayofmonth("data")) \
                   .withColumn("mes", month("data")) \
                   .withColumn("ano", year("data")) \
                   .withColumn("nome_mes", date_format("data", "MMMM")) \
                   .withColumn("dia_da_semana", date_format("data", "EEEE")) \
                   .withColumn("trimestre", quarter("data"))

# (Opcional) Forçar locale pt_BR se Spark estiver configurado
# Isso depende do locale configurado no ambiente do cluster.
# Ex: spark.conf.set("spark.sql.session.timeZone", "America/Sao_Paulo")

# Salvar como tabela Delta (Silver Layer)
df_tempo.write.format("delta").mode("overwrite").saveAsTable("s_dim_tempo")



In [ ]:


# 1) Ler as duas tabelas Delta
df_alucar = spark.read.format("delta").load("Tables/b_despesas_alucar") \
    .select(col("DESPESAS").alias("nome_despesa"), col("Valor").alias("valor"), col("Mês").alias("mes")) \
    .withColumn("origem", lit("Alucar"))

df_consig = spark.read.format("delta").load("Tables/b_despesas_consigcar") \
    .select(col("DESPESAS").alias("nome_despesa"), col("Valor").alias("valor"), col("Mês").alias("mes")) \
    .withColumn("origem", lit("Consigcar"))

# 2) Concatenar
df_desp = df_alucar.unionByName(df_consig)

# 3) Limpar coluna valor: remove 'R$', pontos, troca vírgula por ponto, trim, converte para float
df_desp = df_desp.withColumn(
    "valor",
    regexp_replace(regexp_replace(regexp_replace(trim(col("valor")), "R\\$", ""), "\\.", ""), ",", ".").cast(FloatType())
)

# 4) Criar função para mapear mês em português para número
meses = {
    'janeiro': '01', 'fevereiro': '02', 'março': '03', 'abril': '04',
    'maio': '05', 'junho': '06', 'julho': '07', 'agosto': '08',
    'setembro': '09', 'outubro': '10', 'novembro': '11', 'dezembro': '12'
}

def converte_mes_udf(mes):
    if mes is None:
        return None
    mes = mes.lower().strip()
    for nome, numero in meses.items():
        if mes.startswith(nome):
            return f"2025-{numero}-01"
    return None

udf_converte_mes = udf(converte_mes_udf, StringType())

df_desp = df_desp.withColumn("data", udf_converte_mes(col("mes")))

# 5) Converter para data e depois gerar id_data YYYYMMDD (inteiro)
df_desp = df_desp.withColumn("data_dt", to_date(col("data"), "yyyy-MM-dd"))
df_desp = df_desp.withColumn("id_data", date_format(col("data_dt"), "yyyyMMdd").cast(IntegerType()))

# 6) Criar coluna categoria - você precisa definir a função classificar_despesa para PySpark
# Se for simples, pode fazer um UDF, exemplo abaixo com placeholder

udf_classificar = udf(classificar_despesa, StringType())

df_desp = df_desp.withColumn("categoria", udf_classificar(col("nome_despesa")))

window_spec = Window.orderBy("nome_despesa")
df_despesa_com_id = df_desp.withColumn("id_despesa", row_number().over(window_spec) - 1)

colunas_existentes = [col for col in df_despesa_com_id.columns if col != "id_despesa"]

# Cria a lista final de colunas com 'id_cliente' na primeira posição
colunas_finais = ["id_despesa"] + colunas_existentes

# Seleciona as colunas na ordem desejada
df_despesa_final = df_despesa_com_id.select(*colunas_finais).sort("id_despesa")

# 7) Selecionar colunas para salvar
df_final = df_despesa_final.select("id_despesa", "origem", "categoria", "nome_despesa", "valor", "id_data")

# 8) Salvar na camada silver como Delta
df_final.write.format("delta").mode("overwrite").save("Tables/s_fato_despesas")


In [ ]:
# 1) Ler a tabela Delta
df_pag = spark.read.format("delta").load("Tables/b_receita_pagseguro_consigcar") \
    .select(col("Data").alias("data"), col("Valor").alias("valor_faturado"))

# 2) Converter data para id_data no formato YYYYMMDD inteiro
df_pag = df_pag.withColumn("data_dt", to_date(col("data"), "yyyy-MM-dd"))  # Ajuste o formato se necessário
df_pag = df_pag.withColumn("id_data", date_format(col("data_dt"), "yyyyMMdd").cast(IntegerType()))

# 3) Limpar e converter valor_faturado para float
df_pag = df_pag.withColumn(
    "valor_faturado",
    regexp_replace(regexp_replace(regexp_replace(trim(col("valor_faturado")), "R\\$", ""), "\\.", ""), ",", ".").cast(FloatType())
)

window_spec = Window.orderBy("id_data")
df_pag = df_pag.withColumn("id_faturamento", row_number().over(window_spec) - 1)

colunas_existentes = [col for col in df_pag.columns if col != "id_faturamento"]

# Cria a lista final de colunas com 'id_cliente' na primeira posição
colunas_finais = ["id_faturamento"] + colunas_existentes

# Seleciona as colunas na ordem desejada
df_pag_final = df_pag.select(*colunas_finais).sort("id_faturamento")

# 4) Selecionar colunas para salvar
df_final = df_pag_final.select("id_faturamento","id_data", "valor_faturado")


# Corrigir os id_data que não terminam com '01'
df_final = df_final.withColumn(
    "id_data",
    when(col("id_data") % 100 != 1,  # verifica se os dois últimos dígitos são diferentes de 01
         expr("cast(int(id_data / 100) * 100 + 1 as int)")  # troca os dois últimos dígitos por 01
    ).otherwise(col("id_data"))  # mantém se já termina com 01
)

# 5) Salvar na camada silver (Delta)
df_final.write.format("delta").mode("overwrite").save("Tables/s_fato_faturamento_pagseguro")


In [ ]:
# Definindo a janela ordenada para o row_number
window_spec = Window.orderBy("Data", "Nome__Alucar_")

# 1. Ler a tabela Delta com os dados da Alucar
df_val = (
    spark.read.format("delta")
    .load("Tables/b_vendas_clientes_alucar")
    .select("Data", "Valor_Receita", "Nome__Alucar_")
    .withColumn("id_venda_alucar", row_number().over(window_spec))
    .withColumnRenamed("Data", "data")
    .withColumnRenamed("Valor_Receita", "valor_venda")
    .withColumnRenamed("Nome__Alucar_", "nome_cliente")
)

# 2. Ler a dimensão cliente
df_clientes = (
    spark.read.format("delta")
    .load("Tables/s_dim_cliente")
    .select("id_cliente", "nome_cliente")
)

# 3. Juntar com dim_cliente
df_val = df_val.join(df_clientes, on="nome_cliente", how="left")

# 4. Criar id_data no formato YYYYMMDD
df_val = df_val.withColumn("data_formatada", to_date("data", "yyyy-MM-dd"))
df_val = df_val.withColumn("id_data", date_format("data_formatada", "yyyyMMdd").cast(IntegerType()))

# 5. Converter valor_venda de R$ X.XXX,XX para float
df_val = (
    df_val.withColumn("valor_venda", regexp_replace("valor_venda", "R\\$", ""))
           .withColumn("valor_venda", regexp_replace("valor_venda", "\\.", ""))
           .withColumn("valor_venda", regexp_replace("valor_venda", ",", "."))
           .withColumn("valor_venda", col("valor_venda").cast("float"))
)

# 6. Selecionar colunas finais
df_val_final = df_val.select("id_venda_alucar", "id_cliente", "id_data", "valor_venda")

# 7. Salvar como Delta
df_val_final.write.format("delta").mode("overwrite").save("Tables/s_fato_vendas_alucar")


In [ ]:
# 1) Leitura da tabela 00_vendas_clientes_consigcar da camada bronze
df_cons = (
    spark.read.format("delta")
    .load("Tables/b_vendas_clientes_consigcar")
    .selectExpr(
        "row_number() OVER (ORDER BY Nome) as id_venda_consigcar",  # cria id sequencial (se não existir id)
        "Nome as nome_cliente",
        "`Tipo_Produto` as tipo_produto",
        "`Quantidade_de_vezes` as num_parcelas",
        "`Valor_parcela` as valor_parcela",
        "Vendedor as nome_vendedor",
        "`Data_do_pagamento` as data_primeira_parcela"
    )
)

# Se a tabela já tem uma chave única (ex: rowid), use ela para id_venda_consigcar no lugar do row_number

# 2) Calcular valor_total = num_parcelas * valor_parcela
df_cons = df_cons.withColumn("valor_total", col("num_parcelas") * col("valor_parcela"))

# 3) Converter data_primeira_parcela para date
df_cons = df_cons.withColumn("data_primeira_parcela", to_date("data_primeira_parcela", "yyyy-MM-dd"))

# 4) Calcular data_ultima_parcela = data_primeira_parcela + (num_parcelas - 1) meses
# Spark não tem diretamente add months com col * int, então usamos expr com INTERVAL e SQL
df_cons = df_cons.withColumn(
    "data_ultima_parcela",
    expr("add_months(data_primeira_parcela, CAST(num_parcelas AS int) - 1)")
)

# 5) Formatar as datas no formato YYYYMMDD como int
df_cons = df_cons.withColumn(
    "data_primeira_parcela", date_format("data_primeira_parcela", "yyyyMMdd").cast(IntegerType())
).withColumn(
    "data_ultima_parcela", date_format("data_ultima_parcela", "yyyyMMdd").cast(IntegerType())
)

# 6) Ler dim_cliente para obter id_cliente
df_clientes = spark.read.format("delta").load("Tables/s_dim_cliente").select("id_cliente", "nome_cliente")

# 7) Ler dim_vendedor para obter id_vendedor
df_vend = spark.read.format("delta").load("Tables/s_dim_vendedor").select("id_vendedor", "nome_vendedor")

# 8) Fazer join para incluir id_cliente e id_vendedor
df_cons = df_cons.join(df_clientes, on="nome_cliente", how="left")
df_cons = df_cons.join(df_vend, on="nome_vendedor", how="left")

# 9) Selecionar colunas relevantes
colunas_fato = [
    "id_venda_consigcar", "id_cliente", "tipo_produto", "id_vendedor",
    "num_parcelas", "valor_parcela", "valor_total",
    "data_primeira_parcela", "data_ultima_parcela"
]
df_fato = df_cons.select(colunas_fato)

# 10) Salvar tabela fato na camada silver
df_fato.write.format("delta").mode("overwrite").save("Tables/s_fato_vendas_consigcar")


In [ ]:
# 1) Ler fato_vendas_consigcar da camada silver
df_vendas = spark.read.format("delta").load("Tables/s_fato_vendas_consigcar")

# 2) Ler dim_tempo da camada silver
df_tempo = spark.read.format("delta").load("Tables/s_dim_tempo")

# 3) Fazer join entre fato_vendas_consigcar e dim_tempo
# Aqui usamos cast para comparar os IDs corretamente
df_joined = df_vendas.join(
    df_tempo,
    df_vendas["data_primeira_parcela"].cast("string") == df_tempo["id_data"],
    how="inner"
)

# 4) Agregar: total de vendas e soma de valor_total por id_vendedor e id_data
df_vendas_diaria = (
    df_joined.groupBy("id_vendedor", "id_data","valor_parcela")
    .agg(
        count("*").alias("total_vendas"),
        spark_sum("valor_total").alias("valor_total")
    )
)

# 5) Salvar a tabela na camada silver
df_vendas_diaria.write.format("delta").mode("overwrite").save("Tables/s_fato_vendas_diaria_vendedor")


In [ ]:
# 1. Carregar os dados da fato_vendas_consigcar
df_vendas = spark.read.format("delta").load("Tables/s_fato_vendas_consigcar")

# 2. Converter data_primeira_parcela (formato Int YYYYMMDD) para data real
df_vendas = df_vendas.withColumn(
    "data_inicial",
    to_date(col("data_primeira_parcela").cast("string"), "yyyyMMdd")
)

# 3. Criar a sequência de datas mensais para cada parcela
df_vendas = df_vendas.withColumn(
    "datas_pagamento",
    sequence(
        col("data_inicial"),
        expr("add_months(data_inicial, num_parcelas - 1)"),
        expr("interval 1 month")
    )
)

# 4. Explodir as datas e formatar como id_data (YYYYMMDD)
df_pagamentos = df_vendas.select(
    col("id_venda_consigcar"),
    col("valor_parcela").alias("valor"),
    explode("datas_pagamento").alias("data_pagamento")
).withColumn(
    "id_data", date_format(col("data_pagamento"), "yyyyMMdd").cast(IntegerType())
).select("id_venda_consigcar", "valor", "id_data")

# 5. Salvar como tabela Delta na camada silver
df_pagamentos.write.format("delta").mode("overwrite").save("Tables/s_dim_pagamentos_programados")


In [ ]:
# Inicializa a SparkSession (se ainda não o fez)
spark = SparkSession.builder.appName("MetasAlucar").getOrCreate()

# 1. Carregar os dados da tabela "00_metas_plr"
df_metas = spark.read.format("delta").load("Tables/b_metas_plr")

# 2. Converter a coluna Data para id_data (formato Int: YYYYMMDD)
df_metas = df_metas.withColumn("id_data", date_format(to_date(col("Data")), "yyyyMMdd").cast("int"))

# 3. Selecionar e renomear colunas
df_metas = df_metas.select(
    "id_data",
    col("Meta_1_ALUCAR").alias("meta_vendas_1_cum"),
    col("Meta_2_ALUCAR").alias("meta_vendas_2_cum")
)


# 4. Calcular metas mensais com .diff() via função lag
#    Definindo default=0 para 'lag()' para que a primeira linha de 'meta_vendas_X_mes'
#    seja igual ao 'meta_vendas_X_cum' correspondente.
window_spec = Window.orderBy("id_data")

df_metas = df_metas.withColumn(
    "meta_vendas_1_mes",
    col("meta_vendas_1_cum") - lag("meta_vendas_1_cum", 1, 0).over(window_spec) # <-- Default agora é 0
).withColumn(
    "meta_vendas_2_mes",
    col("meta_vendas_2_cum") - lag("meta_vendas_2_cum", 1, 0).over(window_spec) # <-- Default agora é 0
)

# ---

# 5. Para o primeiro mês do ano (janeiro), usar o valor cumulativo diretamente
#    Esta lógica ainda é útil se suas metas cumulativas resetam anualmente.
df_metas = df_metas.withColumn(
    "mes",
    date_format(col("id_data").cast("string"), "MMdd").substr(1, 2)
)

df_metas = df_metas.withColumn(
    "meta_vendas_1_mes",
    when(col("mes") == "01", col("meta_vendas_1_cum")).otherwise(col("meta_vendas_1_mes"))
).withColumn(
    "meta_vendas_2_mes",
    when(col("mes") == "01", col("meta_vendas_2_cum")).otherwise(col("meta_vendas_2_mes"))
).drop("mes")

# ---

# 6. Salvar a tabela como Delta na camada Silver
df_metas.write.format("delta").mode("overwrite").save("Tables/s_fato_metas_alucar")

In [ ]:
# Initialize SparkSession if not already done
spark = SparkSession.builder.appName("MetasConsigcar").getOrCreate()

# 1. Load data from the "00_metas_plr" table
df_metas_consig = spark.read.format("delta").load("Tables/b_metas_plr")

# 2. Convert the 'Data' column to 'id_data' (YYYYMMDD int)
df_metas_consig = df_metas_consig.withColumn(
    "id_data",
    date_format(col("Data"), "yyyyMMdd").cast("int")
)

# 3. Select and rename columns
df_metas_consig = df_metas_consig.select(
    "id_data",
    col("Meta_1_ConsigCar").alias("meta_vendas_1_cum"),
    col("Meta_2_ConsigCar").alias("meta_vendas_2_cum")
)


# 4. Calculate monthly targets using lag (difference)
#    Setting default=0 for lag ensures that the first row's 'mes' value
#    is equal to its 'cum' value (cum - 0 = cum).
window_spec = Window.orderBy("id_data")

df_metas_consig = df_metas_consig.withColumn(
    "meta_vendas_1_mes",
    col("meta_vendas_1_cum") - lag("meta_vendas_1_cum", 1, 0).over(window_spec) # <-- Default agora é 0
).withColumn(
    "meta_vendas_2_mes",
    col("meta_vendas_2_cum") - lag("meta_vendas_2_cum", 1, 0).over(window_spec) # <-- Default agora é 0
)

# ---

# 5. Adjust for the first month of EACH YEAR (January)
#    This logic will override the above calculation if it's January and
#    you want the monthly value to be the cumulative for that start of year.
df_metas_consig = df_metas_consig.withColumn(
    "mes",
    date_format(col("id_data").cast("string"), "MMdd").substr(1, 2)
)

df_metas_consig = df_metas_consig.withColumn(
    "meta_vendas_1_mes",
    when(col("mes") == "01", col("meta_vendas_1_cum")).otherwise(col("meta_vendas_1_mes"))
).withColumn(
    "meta_vendas_2_mes",
    when(col("mes") == "01", col("meta_vendas_2_cum")).otherwise(col("meta_vendas_2_mes"))
).drop("mes")

# ---

# 6. Save the resulting table to the Silver layer (Delta format)
df_metas_consig.write.format("delta").mode("overwrite").save("Tables/s_fato_metas_consigcar")

In [ ]:
# 1. Carregar os dados da tabela "00_vendas_clientes_alucar_estimativa"
df_vendas_estimativa = spark.read.format("delta").load("Tables/b_vendas_clientes_alucar_estimativa")

# 2. Converter a coluna Data para id_data (YYYYMMDD int)
df_vendas_estimativa = df_vendas_estimativa.withColumn(
    "id_data",
    date_format(col("Data"), "yyyyMMdd").cast("int")
)

# 3. Selecionar e renomear colunas
df_vendas_estimativa = df_vendas_estimativa.select(
    "id_data",
    col("Nome__Alucar_").alias("nome"),
    col("Valor_Receita").alias("valor_receita_estimativa")
)

# 4. Remover 'R$', pontos e substituir vírgula por ponto para conversão numérica
df_vendas_estimativa = df_vendas_estimativa.withColumn(
    "valor_receita_estimativa",
    regexp_replace(col("valor_receita_estimativa"), "R\\$", "")  # Remove 'R$'
)

df_vendas_estimativa = df_vendas_estimativa.withColumn(
    "valor_receita_estimativa",
    regexp_replace(col("valor_receita_estimativa"), "\\.", "")  # Remove pontos de milhar
)

df_vendas_estimativa = df_vendas_estimativa.withColumn(
    "valor_receita_estimativa",
    regexp_replace(col("valor_receita_estimativa"), ",", ".")  # Troca vírgula por ponto decimal
)

# 5. Converter para float
df_vendas_estimativa = df_vendas_estimativa.withColumn(
    "valor_receita_estimativa",
    col("valor_receita_estimativa").cast("float")
)

# 6. Salvar a tabela no formato Delta ou conforme sua arquitetura
df_vendas_estimativa.write.format("delta").mode("overwrite").save("Tables/s_fato_vendas_clientes_alucar_estimativa")


In [ ]:
# 1. Carregar os dados da tabela "00_receita_consigcar_estimativa"
df_consig_estimativa = spark.read.format("delta").load("Tables/b_receita_consigcar_estimativa")

# 2. Converter a coluna Data para id_data no formato YYYYMMDD (inteiro)
df_consig_estimativa = df_consig_estimativa.withColumn(
    "id_data",
    date_format(col("Data"), "yyyyMMdd").cast("int")
)

# 3. Selecionar e renomear colunas
df_consig_estimativa = df_consig_estimativa.select(
    "id_data",
    col("Valor").alias("valor_receita_estimativa")
)

# 4. Remover 'R$', pontos e substituir vírgula por ponto para conversão numérica
df_consig_estimativa = df_consig_estimativa.withColumn(
    "valor_receita_estimativa",
    regexp_replace(col("valor_receita_estimativa"), "R\\$", "")
)

df_consig_estimativa = df_consig_estimativa.withColumn(
    "valor_receita_estimativa",
    regexp_replace(col("valor_receita_estimativa"), "\\.", "")
)

df_consig_estimativa = df_consig_estimativa.withColumn(
    "valor_receita_estimativa",
    regexp_replace(col("valor_receita_estimativa"), ",", ".")
)

# 5. Converter para float
df_consig_estimativa = df_consig_estimativa.withColumn(
    "valor_receita_estimativa",
    col("valor_receita_estimativa").cast("float")
)

# 6. Salvar a tabela no formato Delta ou conforme sua arquitetura
df_consig_estimativa.write.format("delta").mode("overwrite").save("Tables/s_fato_consigcar_estimativa")
